# Algo trading simluation with SMA

1. Define a SMA period.
2. If the adjusted close price is above the SMA(SMA period), enter a long position.
3. If a long position is in place, and the adjusted close price drops below the SMA(15), exit the long position.

In [ ]:
!pip install pyalgotrade
#!pip install --upgrade pandas
!pip install --upgrade pandas==1.5.3
!pip install --upgrade pandas-datareader
!pip install yfinance


In [ ]:
from __future__ import print_function

from pyalgotrade import strategy
from pyalgotrade.barfeed import yahoofeed
from pyalgotrade.technical import ma

import datetime as dt
import pandas as pd
import pandas_datareader.data as web
import yfinance as yf
import matplotlib.pyplot as plt

print(
    f"Pandas {pd.__version__}"
)

In [ ]:
# Data collection

yf.pdr_override()
start = dt.datetime(2020,1,1)
end = dt.datetime(2022,4,21)
query_str = "AAPL"
df = web.get_data_yahoo(query_str, start, end)

filename = f'{query_str}.csv'
df.to_csv(filename)

In [ ]:
# Create strategy test

class StrategyTemplate(strategy.BacktestingStrategy):
    """
    Class to create a strategy template with the SMA strategy
    
    :param feed: historical price data to be fed for backtesting
    :param instrument: the asset/ the security
    :param smaPeriod: period of the SMA you would like use
    """
    def __init__(self, feed, instrument, smaPeriod):   
        """
        Initialization for the strategy
        """
        super(StrategyTemplate, self).__init__(feed, 1000)
        self.__position = None
        self.__instrument = instrument
        self.setUseAdjustedValues(True)
        self.__sma = ma.SMA(feed[instrument].getPriceDataSeries(), smaPeriod)

    def onEnterOk(self, position):    
        """
        Method for actions if the program enters a position
        """
        execInfo = position.getEntryOrder().getExecutionInfo()
        self.info("BUY at ${}".format(execInfo.getPrice()))
  
    def onEnterCanceled(self, position):        
        """
        Method for actions if the program cancels entering the position
        """
        self.__position = None
  
    def onExitOk(self, position):                           
        """
        Method for actions if the program exits a position
        """
        execInfo = position.getExitOrder().getExecutionInfo()
        self.info("SELL at ${}".format(execInfo.getPrice()))
        self.__position  = None

    def onExitCancelled(self, position):            
        """
        Method for actions if the program cancels exiting a position
        """
        self.__position.exitMarket()
  
    def onBars(self, bars):            
        """
        Method for actions after reading each row
        """
        if self.__sma[-1] is None:
            return

        bar = bars[self.__instrument]

        if self.__position is None:
            if bar.getPrice() > self.__sma[-1]:
                self.__position = self.enterLong(self.__instrument, 10, True)
    
        elif bar.getPrice() < self.__sma[-1] and not self.__position.exitActive():
            self.__position.exitMarket()

In [ ]:
# Run strategy

def run_strategy(smaPeriod):
    feed = yahoofeed.Feed()
    feed.addBarsFromCSV(query_str, filename)

    myStrategy = StrategyTemplate(feed, query_str, smaPeriod)
    myStrategy.run()
    print("Final portfolio value: ${}".format(myStrategy.getBroker().getEquity()))
    return myStrategy.getResult()

strategy_results = []
labels = []
for i in range(10,30):
    strategy_results.append(run_strategy(i))
    labels.append(i)

In [ ]:
# Visualisation

plt.figure(figsize=(20,7))
plt.bar(labels, strategy_results)
plt.xticks(labels)